In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from rapidfuzz import process

import csv
import string
from sqlalchemy import create_engine
import sqlite3

In [2]:
BD = pd.read_csv('COVID-19 Activity.csv', low_memory = False)
BD.set_index('index', inplace = True)

In [3]:
BD['CONTINENT_NAME'].fillna(value = 'Others', inplace = True)
BD['COUNTY_NAME'].fillna(value = 'Not Provided', inplace = True)
BD['PROVINCE_STATE_NAME'].fillna(value = 'Not Provided', inplace = True)
BD['CONTINENT_NAME'].fillna(value = 'Not Provided', inplace = True)
BD['COUNTY_FIPS_NUMBER'].fillna(value = '0.0', inplace = True)

In [4]:
Cont_Uni_ID = pd.DataFrame({'CONTINENT_NAME': BD['CONTINENT_NAME'].unique()})
Cont_Uni_ID['Cont_ID'] = Cont_Uni_ID['CONTINENT_NAME'].str[:2].str.upper() + pd.RangeIndex(start = 10, stop = 16, step = 1).astype(str)

Cont_Uni_ID.set_index(['CONTINENT_NAME'], inplace = True)

Prov_Uni_ID = pd.DataFrame({'PROVINCE_STATE_NAME' : BD['PROVINCE_STATE_NAME'].unique()})
Prov_Uni_ID['PV_ID'] = Prov_Uni_ID['PROVINCE_STATE_NAME'].str[:2].str.upper() + "_" + pd.RangeIndex(len(BD['PROVINCE_STATE_NAME'].unique())).astype(str)

Prov_Uni_ID.set_index('PROVINCE_STATE_NAME', inplace = True)

Coun_Uni_ID = pd.DataFrame({'COUNTY_NAME' : BD['COUNTY_NAME'].unique()})
Coun_Uni_ID['COTY_ID'] = Coun_Uni_ID['COUNTY_NAME'].str[:2].str.upper() + pd.RangeIndex(len(BD['COUNTY_NAME'].unique())).astype(str)

Coun_Uni_ID.set_index('COUNTY_NAME', inplace = True)

DT_Uni_ID = pd.DataFrame({'DATA_SOURCE_NAME' : BD['DATA_SOURCE_NAME'].unique()})
DT_Uni_ID['DS_ID'] = DT_Uni_ID['DATA_SOURCE_NAME'].str[:2].str.upper() + pd.RangeIndex(len(BD['DATA_SOURCE_NAME'].unique())).astype(str)

DT_Uni_ID.set_index('DATA_SOURCE_NAME', inplace = True)

Co_Uni_ID = pd.DataFrame({'COUNTRY' : BD['COUNTRY'].unique()})
Co_Uni_ID['CTR_ID'] = Co_Uni_ID['COUNTRY'].str[:2].str.upper() + pd.RangeIndex(len(BD['COUNTRY'].unique())).astype(str)

Co_Uni_ID.set_index('COUNTRY', inplace = True)

In [5]:
Big_DS = pd.merge(BD, Cont_Uni_ID, how = 'left', on = 'CONTINENT_NAME')
Big_DS = pd.merge(Big_DS, Prov_Uni_ID, how = 'left', on = 'PROVINCE_STATE_NAME')
Big_DS = pd.merge(Big_DS, Coun_Uni_ID, how = 'left', on = 'COUNTY_NAME')
Big_DS = pd.merge(Big_DS, DT_Uni_ID, how = 'left', on = 'DATA_SOURCE_NAME')
Big_DS = pd.merge(Big_DS, Co_Uni_ID, how = 'left', on = 'COUNTRY')

In [6]:
Coun_alpha1_Uni_ID = pd.DataFrame({'CN_ALP_ID' : BD['COUNTRY_ALPHA_3_CODE'].unique()})
Coun_alpha2_Uni_ID = pd.DataFrame({'CN_ALP_ID' : BD['COUNTRY_ALPHA_2_CODE'].unique()})

Coun_alpha_Uni_ID = Coun_alpha1_Uni_ID + "_" + Coun_alpha2_Uni_ID

In [7]:
Big_DS['BD_ID'] = Big_DS['Cont_ID'] + Big_DS['PV_ID'] + "_" + Big_DS['COTY_ID'] + Big_DS['DS_ID'] + "_" + Big_DS['CTR_ID'] + "_" + Big_DS['COUNTRY_ALPHA_2_CODE'].str[:2].str.upper()  + "-" + Big_DS['COUNTRY_ALPHA_3_CODE'].str[:2].str.upper() 
Big_DS.set_index('BD_ID', inplace = True)
Big_DS.reindex(sorted(Big_DS.columns), axis= 1)

,CONTINENT_NAME,COTY_ID,COUNTRY,COUNTRY_ALPHA_2_CODE,COUNTRY_ALPHA_3_CODE,COUNTY_FIPS_NUMBER,COUNTY_NAME,CTR_ID,Cont_ID,DATA_SOURCE_NAME,DS_ID,PEOPLE_DEATH_COUNT,PEOPLE_DEATH_NEW_COUNT,PEOPLE_POSITIVE_CASES_COUNT,PEOPLE_POSITIVE_NEW_CASES_COUNT,PROVINCE_STATE_NAME,PV_ID,REPORT_DATE
BD_ID,,,,,,,,,,,,,,,,,,
AM10OH_0_DE0NE0_UN0_US-US,America,DE0,United States,US,USA,39041.0,Delaware,UN0,AM10,New York Times,NE0,138,1,19088,11,Ohio,OH_0,23-07-2021
AM10OH_0_DE0NE0_UN0_US-US,America,DE0,United States,US,USA,39041.0,Delaware,UN0,AM10,New York Times,NE0,138,0,19099,11,Ohio,OH_0,24-07-2021
AM10OH_0_DE0NE0_UN0_US-US,America,DE0,United States,US,USA,39041.0,Delaware,UN0,AM10,New York Times,NE0,138,0,19105,6,Ohio,OH_0,25-07-2021
AM10OH_0_DE0NE0_UN0_US-US,America,DE0,United States,US,USA,39041.0,Delaware,UN0,AM10,New York Times,NE0,138,0,19120,15,Ohio,OH_0,26-07-2021
AM10OH_0_DE0NE0_UN0_US-US,America,DE0,United States,US,USA,39041.0,Delaware,UN0,AM10,New York Times,NE0,138,0,19143,23,Ohio,OH_0,27-07-2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AM10CA_26_BU861NE0_UN0_US-US,America,BU861,United States,US,USA,6007.0,Butte,UN0,AM10,New York Times,NE0,380,0,39396,7,California,CA_26,11-03-2022
AM10CA_26_BU861NE0_UN0_US-US,America,BU861,United States,US,USA,6007.0,Butte,UN0,AM10,New York Times,NE0,380,0,39396,0,California,CA_26,12-03-2022
AM10CA_26_BU861NE0_UN0_US-US,America,BU861,United States,US,USA,6007.0,Butte,UN0,AM10,New York Times,NE0,380,0,39396,0,California,CA_26,13-03-2022


In [8]:
Big_DS['Day'] = pd.to_datetime(Big_DS['REPORT_DATE'], dayfirst = True).dt.day
Big_DS['Month'] = pd.to_datetime(Big_DS['REPORT_DATE'], dayfirst = True).dt.month
Big_DS['Year'] = pd.to_datetime(Big_DS['REPORT_DATE'], dayfirst = True).dt.year
Big_DS['Quarter'] = pd.to_datetime(Big_DS['REPORT_DATE'], dayfirst = True).dt.quarter

Big_DS.fillna(value = '0.0', inplace = True)

In [9]:
Am_Data = Big_DS[Big_DS['CONTINENT_NAME'] == 'America']
Af_Data = Big_DS[Big_DS['CONTINENT_NAME'] == 'Africa']
As_Data = Big_DS[Big_DS['CONTINENT_NAME'] == 'Asia']
Ep_Data = Big_DS[Big_DS['CONTINENT_NAME'] == 'Europe']
On_Data = Big_DS[Big_DS['CONTINENT_NAME'] == 'Oceania']
Ot_Data = Big_DS[Big_DS['CONTINENT_NAME'] == 'Others']

In [10]:
Big_DS.to_csv('Big_DS.csv', index = True)
Am_Data.to_csv('Am_Data.csv', index = True)
Af_Data.to_csv('Af_Data.csv', index = True)
As_Data.to_csv('As_Data.csv', index = True)
Ep_Data.to_csv('Ep_Data.csv', index = True)
On_Data.to_csv('On_Data.csv', index = True)
Ot_Data.to_csv('Ot_Data.csv', index = True)

In [11]:
Big_DS['DPC_Ratio'] = round((Big_DS['PEOPLE_DEATH_COUNT'] / Big_DS['PEOPLE_POSITIVE_CASES_COUNT'] * 100), 2)
Big_DS['New_DPC_Ratio'] = round((Big_DS['PEOPLE_DEATH_NEW_COUNT'] / Big_DS['PEOPLE_POSITIVE_NEW_CASES_COUNT'] * 100), 2)

In [12]:
Big_DS['New_DPC_Ratio'].fillna(value = '0.0', inplace = True)
Big_DS['DPC_Ratio'].fillna(value = '0.0', inplace = True)

In [13]:
conn = sqlite3.connect(':memory:')

Big_DS.to_sql('Big_DS', conn, index = True, if_exists = 'replace')

1048575

In [14]:
Q1 = """SELECT * FROM Big_DS """

res = pd.read_sql(Q1, conn)
res

,BD_ID,PEOPLE_POSITIVE_CASES_COUNT,COUNTY_NAME,PROVINCE_STATE_NAME,REPORT_DATE,CONTINENT_NAME,DATA_SOURCE_NAME,PEOPLE_DEATH_NEW_COUNT,COUNTY_FIPS_NUMBER,COUNTRY_ALPHA_3_CODE,...,PV_ID,COTY_ID,DS_ID,CTR_ID,Day,Month,Year,Quarter,DPC_Ratio,New_DPC_Ratio
0,AM10OH_0_DE0NE0_UN0_US-US,19088,Delaware,Ohio,23-07-2021,America,New York Times,1,39041.0,USA,...,OH_0,DE0,NE0,UN0,23,7,2021,3,0.72,9.09
1,AM10OH_0_DE0NE0_UN0_US-US,19099,Delaware,Ohio,24-07-2021,America,New York Times,0,39041.0,USA,...,OH_0,DE0,NE0,UN0,24,7,2021,3,0.72,0.0
2,AM10OH_0_DE0NE0_UN0_US-US,19105,Delaware,Ohio,25-07-2021,America,New York Times,0,39041.0,USA,...,OH_0,DE0,NE0,UN0,25,7,2021,3,0.72,0.0
3,AM10OH_0_DE0NE0_UN0_US-US,19120,Delaware,Ohio,26-07-2021,America,New York Times,0,39041.0,USA,...,OH_0,DE0,NE0,UN0,26,7,2021,3,0.72,0.0
4,AM10OH_0_DE0NE0_UN0_US-US,19143,Delaware,Ohio,27-07-2021,America,New York Times,0,39041.0,USA,...,OH_0,DE0,NE0,UN0,27,7,2021,3,0.72,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,AM10CA_26_BU861NE0_UN0_US-US,39396,Butte,California,11-03-2022,America,New York Times,0,6007.0,USA,...,CA_26,BU861,NE0,UN0,11,3,2022,1,0.96,0.0
1048571,AM10CA_26_BU861NE0_UN0_US-US,39396,Butte,California,12-03-2022,America,New York Times,0,6007.0,USA,...,CA_26,BU861,NE0,UN0,12,3,2022,1,0.96,0.0
1048572,AM10CA_26_BU861NE0_UN0_US-US,39396,Butte,California,13-03-2022,America,New York Times,0,6007.0,USA,...,CA_26,BU861,NE0,UN0,13,3,2022,1,0.96,0.0
1048573,AM10CA_26_BU861NE0_UN0_US-US,39428,Butte,California,14-03-2022,America,New York Times,4,6007.0,USA,...,CA_26,BU861,NE0,UN0,14,3,2022,1,0.97,12.5
